# Adams-Bashforth Methods for Numerical Solution of ODEs

## Introduction

The **Adams-Bashforth methods** are a family of explicit linear multistep methods used to solve ordinary differential equations (ODEs) of the form:

$$\frac{dy}{dt} = f(t, y), \quad y(t_0) = y_0$$

Unlike single-step methods such as Euler or Runge-Kutta, multistep methods utilize information from several previous time steps to achieve higher accuracy without increasing the number of function evaluations per step.

## Derivation

The Adams-Bashforth methods are derived by integrating the ODE from $t_n$ to $t_{n+1}$:

$$y_{n+1} = y_n + \int_{t_n}^{t_{n+1}} f(t, y(t)) \, dt$$

The key idea is to approximate $f(t, y(t))$ by a polynomial that interpolates the known values $f_n, f_{n-1}, f_{n-2}, \ldots$ at previous time steps.

### Adams-Bashforth Formulas

**First-Order (AB1) - Explicit Euler:**
$$y_{n+1} = y_n + h f_n$$

**Second-Order (AB2):**
$$y_{n+1} = y_n + \frac{h}{2}(3f_n - f_{n-1})$$

**Third-Order (AB3):**
$$y_{n+1} = y_n + \frac{h}{12}(23f_n - 16f_{n-1} + 5f_{n-2})$$

**Fourth-Order (AB4):**
$$y_{n+1} = y_n + \frac{h}{24}(55f_n - 59f_{n-1} + 37f_{n-2} - 9f_{n-3})$$

### Local Truncation Error

The local truncation error for an $s$-step Adams-Bashforth method is $\mathcal{O}(h^{s+1})$, giving global error of $\mathcal{O}(h^s)$.

### Stability

Adams-Bashforth methods have limited stability regions. For a test equation $y' = \lambda y$ with $\lambda < 0$, the stability constraint becomes more restrictive as the order increases.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.linewidth'] = 2

## Implementation of Adams-Bashforth Methods

We implement Adams-Bashforth methods of orders 1 through 4. Since these are multistep methods, we need starting values which we obtain using the classical fourth-order Runge-Kutta method.

In [ ]:
def runge_kutta_4(f, t, y, h):
    """Fourth-order Runge-Kutta method for a single step."""
    k1 = f(t, y)
    k2 = f(t + h/2, y + h*k1/2)
    k3 = f(t + h/2, y + h*k2/2)
    k4 = f(t + h, y + h*k3)
    return y + h * (k1 + 2*k2 + 2*k3 + k4) / 6

def adams_bashforth_1(f, t_span, y0, h):
    """First-order Adams-Bashforth (Explicit Euler)."""
    t0, tf = t_span
    t = np.arange(t0, tf + h, h)
    n = len(t)
    y = np.zeros(n)
    y[0] = y0
    
    for i in range(n - 1):
        y[i+1] = y[i] + h * f(t[i], y[i])
    
    return t, y

def adams_bashforth_2(f, t_span, y0, h):
    """Second-order Adams-Bashforth method."""
    t0, tf = t_span
    t = np.arange(t0, tf + h, h)
    n = len(t)
    y = np.zeros(n)
    y[0] = y0
    
    # Starting value using RK4
    y[1] = runge_kutta_4(f, t[0], y[0], h)
    
    for i in range(1, n - 1):
        f_n = f(t[i], y[i])
        f_nm1 = f(t[i-1], y[i-1])
        y[i+1] = y[i] + h * (3*f_n - f_nm1) / 2
    
    return t, y

def adams_bashforth_3(f, t_span, y0, h):
    """Third-order Adams-Bashforth method."""
    t0, tf = t_span
    t = np.arange(t0, tf + h, h)
    n = len(t)
    y = np.zeros(n)
    y[0] = y0
    
    # Starting values using RK4
    for i in range(2):
        y[i+1] = runge_kutta_4(f, t[i], y[i], h)
    
    for i in range(2, n - 1):
        f_n = f(t[i], y[i])
        f_nm1 = f(t[i-1], y[i-1])
        f_nm2 = f(t[i-2], y[i-2])
        y[i+1] = y[i] + h * (23*f_n - 16*f_nm1 + 5*f_nm2) / 12
    
    return t, y

def adams_bashforth_4(f, t_span, y0, h):
    """Fourth-order Adams-Bashforth method."""
    t0, tf = t_span
    t = np.arange(t0, tf + h, h)
    n = len(t)
    y = np.zeros(n)
    y[0] = y0
    
    # Starting values using RK4
    for i in range(3):
        y[i+1] = runge_kutta_4(f, t[i], y[i], h)
    
    for i in range(3, n - 1):
        f_n = f(t[i], y[i])
        f_nm1 = f(t[i-1], y[i-1])
        f_nm2 = f(t[i-2], y[i-2])
        f_nm3 = f(t[i-3], y[i-3])
        y[i+1] = y[i] + h * (55*f_n - 59*f_nm1 + 37*f_nm2 - 9*f_nm3) / 24
    
    return t, y

## Test Problem: Exponential Decay

We first test our implementations on the simple exponential decay equation:

$$\frac{dy}{dt} = -y, \quad y(0) = 1$$

The exact solution is $y(t) = e^{-t}$.

In [ ]:
# Define the test problem
def f_decay(t, y):
    return -y

def exact_decay(t):
    return np.exp(-t)

# Parameters
t_span = (0, 5)
y0 = 1.0
h = 0.1

# Solve using all methods
t1, y1 = adams_bashforth_1(f_decay, t_span, y0, h)
t2, y2 = adams_bashforth_2(f_decay, t_span, y0, h)
t3, y3 = adams_bashforth_3(f_decay, t_span, y0, h)
t4, y4 = adams_bashforth_4(f_decay, t_span, y0, h)

# Exact solution
t_exact = np.linspace(0, 5, 200)
y_exact = exact_decay(t_exact)

## Convergence Analysis

To verify the order of accuracy, we compute the global error for different step sizes and observe the convergence rate.

In [ ]:
def compute_errors(method, f, exact, t_span, y0, step_sizes):
    """Compute maximum errors for different step sizes."""
    errors = []
    for h in step_sizes:
        t, y = method(f, t_span, y0, h)
        y_true = exact(t)
        error = np.max(np.abs(y - y_true))
        errors.append(error)
    return np.array(errors)

# Step sizes for convergence study
step_sizes = np.array([0.2, 0.1, 0.05, 0.025, 0.0125])

# Compute errors for each method
errors_ab1 = compute_errors(adams_bashforth_1, f_decay, exact_decay, t_span, y0, step_sizes)
errors_ab2 = compute_errors(adams_bashforth_2, f_decay, exact_decay, t_span, y0, step_sizes)
errors_ab3 = compute_errors(adams_bashforth_3, f_decay, exact_decay, t_span, y0, step_sizes)
errors_ab4 = compute_errors(adams_bashforth_4, f_decay, exact_decay, t_span, y0, step_sizes)

## Application: Damped Harmonic Oscillator

Consider the damped harmonic oscillator:

$$\frac{d^2x}{dt^2} + 2\zeta\omega_0\frac{dx}{dt} + \omega_0^2 x = 0$$

Converting to a first-order system with $y_1 = x$ and $y_2 = dx/dt$:

$$\frac{dy_1}{dt} = y_2$$
$$\frac{dy_2}{dt} = -2\zeta\omega_0 y_2 - \omega_0^2 y_1$$

In [ ]:
def adams_bashforth_4_system(f, t_span, y0, h):
    """Fourth-order Adams-Bashforth for systems of ODEs."""
    t0, tf = t_span
    t = np.arange(t0, tf + h, h)
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    
    # RK4 for systems
    def rk4_step(f, t, y, h):
        k1 = np.array(f(t, y))
        k2 = np.array(f(t + h/2, y + h*k1/2))
        k3 = np.array(f(t + h/2, y + h*k2/2))
        k4 = np.array(f(t + h, y + h*k3))
        return y + h * (k1 + 2*k2 + 2*k3 + k4) / 6
    
    # Starting values
    for i in range(3):
        y[i+1] = rk4_step(f, t[i], y[i], h)
    
    for i in range(3, n - 1):
        f_n = np.array(f(t[i], y[i]))
        f_nm1 = np.array(f(t[i-1], y[i-1]))
        f_nm2 = np.array(f(t[i-2], y[i-2]))
        f_nm3 = np.array(f(t[i-3], y[i-3]))
        y[i+1] = y[i] + h * (55*f_n - 59*f_nm1 + 37*f_nm2 - 9*f_nm3) / 24
    
    return t, y

# Damped oscillator parameters
omega0 = 2.0  # Natural frequency
zeta = 0.15   # Damping ratio (underdamped)

def f_oscillator(t, y):
    return [y[1], -2*zeta*omega0*y[1] - omega0**2*y[0]]

# Initial conditions: displaced, zero velocity
y0_osc = [1.0, 0.0]
t_span_osc = (0, 10)
h_osc = 0.05

# Solve
t_osc, y_osc = adams_bashforth_4_system(f_oscillator, t_span_osc, y0_osc, h_osc)

# Analytical solution for underdamped oscillator
omega_d = omega0 * np.sqrt(1 - zeta**2)  # Damped frequency
t_analytical = np.linspace(0, 10, 500)
x_analytical = np.exp(-zeta*omega0*t_analytical) * np.cos(omega_d*t_analytical)

## Visualization

We create a comprehensive visualization showing:
1. Comparison of Adams-Bashforth methods on exponential decay
2. Convergence analysis (log-log plot)
3. Damped oscillator solution
4. Phase portrait of the oscillator

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 11))

# Plot 1: Solutions comparison
ax1 = axes[0, 0]
ax1.plot(t_exact, y_exact, 'k-', label='Exact', linewidth=2.5)
ax1.plot(t1, y1, 'o-', label='AB1', markersize=4, alpha=0.7)
ax1.plot(t2, y2, 's-', label='AB2', markersize=4, alpha=0.7)
ax1.plot(t3, y3, '^-', label='AB3', markersize=4, alpha=0.7)
ax1.plot(t4, y4, 'd-', label='AB4', markersize=4, alpha=0.7)
ax1.set_xlabel('Time $t$')
ax1.set_ylabel('$y(t)$')
ax1.set_title('Exponential Decay: $dy/dt = -y$')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: Convergence analysis
ax2 = axes[0, 1]
ax2.loglog(step_sizes, errors_ab1, 'o-', label='AB1 (1st order)', markersize=8)
ax2.loglog(step_sizes, errors_ab2, 's-', label='AB2 (2nd order)', markersize=8)
ax2.loglog(step_sizes, errors_ab3, '^-', label='AB3 (3rd order)', markersize=8)
ax2.loglog(step_sizes, errors_ab4, 'd-', label='AB4 (4th order)', markersize=8)

# Reference slopes
h_ref = np.array([0.2, 0.0125])
for order, color in [(1, 'C0'), (2, 'C1'), (3, 'C2'), (4, 'C3')]:
    ref = 0.5 * h_ref**order
    ax2.loglog(h_ref, ref, '--', color=color, alpha=0.5)

ax2.set_xlabel('Step size $h$')
ax2.set_ylabel('Maximum Error')
ax2.set_title('Convergence Analysis')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3, which='both')

# Plot 3: Damped oscillator
ax3 = axes[1, 0]
ax3.plot(t_analytical, x_analytical, 'k-', label='Analytical', linewidth=2)
ax3.plot(t_osc, y_osc[:, 0], 'b--', label='AB4 Numerical', linewidth=1.5)
ax3.plot(t_analytical, np.exp(-zeta*omega0*t_analytical), 'r:', 
         label='Envelope $e^{-\\zeta\\omega_0 t}$', linewidth=1.5)
ax3.plot(t_analytical, -np.exp(-zeta*omega0*t_analytical), 'r:', linewidth=1.5)
ax3.set_xlabel('Time $t$')
ax3.set_ylabel('Displacement $x(t)$')
ax3.set_title(f'Damped Harmonic Oscillator ($\\zeta={zeta}$, $\\omega_0={omega0}$)')
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Plot 4: Phase portrait
ax4 = axes[1, 1]
ax4.plot(y_osc[:, 0], y_osc[:, 1], 'b-', linewidth=1.5)
ax4.plot(y_osc[0, 0], y_osc[0, 1], 'go', markersize=10, label='Start')
ax4.plot(y_osc[-1, 0], y_osc[-1, 1], 'ro', markersize=10, label='End')
ax4.set_xlabel('Displacement $x$')
ax4.set_ylabel('Velocity $\\dot{x}$')
ax4.set_title('Phase Portrait')
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)
ax4.axis('equal')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved to plot.png")

## Summary

We have implemented and tested Adams-Bashforth methods of orders 1 through 4. Key observations:

1. **Accuracy**: Higher-order methods achieve significantly better accuracy for the same step size
2. **Convergence**: The log-log convergence plot confirms the theoretical orders of accuracy
3. **Efficiency**: Adams-Bashforth methods require only one function evaluation per step (after startup)
4. **Startup**: Multistep methods require starting values from a single-step method (we used RK4)

### Advantages of Adams-Bashforth Methods
- Computationally efficient (one function evaluation per step)
- Easy to implement
- Explicit formulation (no implicit solves required)

### Limitations
- Require starting values from another method
- Limited stability regions (more restrictive at higher orders)
- Not suitable for stiff problems

For improved stability with similar computational cost, consider the Adams-Moulton (implicit) or predictor-corrector methods.